In [ ]:
import socket
import numpy
import cv2 
import threading

In [ ]:
sk = socket.socket()
sk.bind(("", 4321))  # empty means local system
server_ip = input("enter ip of server: ")
server_port = input("enter port number of server: ")

sk.connect((server_ip, int(server_port))) 
sk.send(b"connected")  # sending string as data
cameraIndex = 0 # laptop webcam
camera = cv2.VideoCapture(cameraIndex) # starting the camera

# function for cleint to work as receiver
def receiver():
    framesLost = 0
    print("Entered")
    while True:
        framesLost += 1 # counting frame
        data = sk.recv(100000000) 
        if(data == b'finished'): # to stop receiving and stop camera
            print("Finished")
            camera.release()
            sk.close()
        else:  # converting the byte data into numpy array
            photo =  numpy.frombuffer(data, dtype=numpy.uint8)
            if len(photo) == 640*480*3: # changing the array shape and getting the video
                cv2.imshow('From Server', photo.reshape(480, 640, 3))
                if cv2.waitKey(1) == 13: # camera closing condition
                    sk.send(b'finished')
                    camera.release()
                    cv2.destroyAllWindows()
                    break
            else:
                print("Lost {} frames".format(framesLost) ) #counting the lost frames during video exchange

# function for client to send the data of the video             
def sender():
    while True: # reading the camera data resizing and sending it as byte 
        status, photo = camera.read()
        photo = cv2.resize(photo, (640, 480))
        #photo = cv.resize(photo, (600, 500))
        print(photo.shape)
        if status:
            sk.send(numpy.ndarray.tobytes(photo))
        else: print("Could not get frame")
    camera.release()

# threads to run both the functions
threading.Thread(target=receiver).start()
threading.Thread(target=sender).start()